## kd Tree

Construct a kd Tree

In [1]:
class KdNode(object):
    def __init__(self, dom_elt, split, left, right):
        self.dom_elt = dom_elt #k维向量节点(k维空间中的一个样本点)
        self.split = split #整数(进行分割维度的序号)
        self.left = left #该节点分割超平面子空间构成的kd-tree
        self.right = right #该节点分割超平面右子空间构成kd-tree
        
        
class kdTree(object):
    def __init__(self, data):
        k = len(data[0])
        
        def CreateNode(split, data_set): #按第split维划分数据集exset创建KdNode
            if not data_set:
                return None
            #key参数的值为一个函数，此函数只有一个参数且返回一个值用来进行比较
            #operator模块提供的itemgetter函数用于获取对象的哪些维的数据，参数为需要获取的数据在对象中的序号
            #data_set.sort(key=itemgetter(split)) #按要进行分割的那一维排序
            #这里data_set是list, 因此使用sort方法排序
            #参数key的作用: key specifies a function of one argument that is used to extract a 
            #comparison key from each element in iterable (for example, key=str.lower). 
            #The default value is None (compare the elements directly).
            
            data_set.sort(key = lambda x: x[split])
            split_pos = len(data_set) // 2
            median = data_set[split_pos]
            split_next = (split+1) % k
            
            return KdNode(
                median,
                split, 
                CreateNode(split_next, data_set[:split_pos]),
                CreateNode(split_next, data_set[split_pos+1:])
            )
        
        self.root = CreateNode(0, data) #从第0维开始构建kd树，返回根节点
        

#kdTree的前序遍历
def preorder(root):
    print(root.dom_elt)
    if root.left != None:
        preorder(root.left)
    if root.right != None:
        preorder(root.right)

In [2]:
# 对构建好的kd树进行搜索，寻找与目标点最近的样本点

from math import sqrt
from collections import namedtuple

# 定义一个namedtuple,分别存放最近坐标点、最近距离和访问过的节点数
result = namedtuple("Result_tuple", "nearest_point nearest_dist nodes_visited")


def find_nearest(tree, point):
    k = len(point) #数据维度
    
    def travel(kd_node, target, max_dist):
        if kd_node is None:
            return result([0] * k, float("inf"), 0)  
        
        nodes_visited = 1
        
        s = kd_node.split  #进行分割的维度
        pivot = kd_node.dom_elt  #进行分割的轴
        
        if target[s] <= pivot[s]: #如果目标点第s维小于分割轴的对应值(目标离左子树更近)
            nearer_node = kd_node.left
            further_node = kd_node.right
        else:
            nearer_node = kd_node.right
            further_node = kd_node.left
        
        temp1 = travel(nearer_node, target, max_dist)  #进行遍历找到包含目标点的区域 搜索到叶节点停止， 准确的说，搜索到叶节点的
        #子节点，因为该节点并不存在，所以返回 空节点: result([0] * k, float("inf"), 0) 然后继续递归  
        
        nearest = temp1.nearest_point  #以此叶节点作为当前最近点
        dist = temp1.nearest_dist #更新最近距离
        
        nodes_visited += temp1.nodes_visited
        
        if dist < max_dist:
            max_dist = dist #最近点将在以目标点为球心， max_dist为半径的超球体内
            
        temp_dist = abs(pivot[s] - target[s])
        if max_dist < temp_dist: #判断超球体是否与超平面相交
            return result(nearest, dist, nodes_visited) #不相交可以直接返回 因为如果不和任何超平面相交，就不可能与同一父节点的另一个子节点距离更近
        
        #------------------------------------------------------
        #计算目标点与分割点的欧氏距离
        temp_dist = sqrt(sum((p1 - p2)**2 for p1, p2 in zip(pivot, target)))
        
        if temp_dist < dist: #如果"更近"
            nearest = pivot
            dist = temp_dist
            max_dist = dist
            
        #检查另一个子节点对应的区域是否有更近的点
        temp2 = travel(further_node, target, max_dist)
        
        nodes_visited += temp2.nodes_visited
        if temp2.nearest_dist < dist: #如果另一个子节点存在更近距离
            nearest = temp2.nearest_point  #更新最近点
            dist = temp.nearest_dist #更新最近距离
            
        return result(nearest, dist, nodes_visited)
        
    return travel(tree.root, point, float("inf")) #从根节点开始递归

例3.2

In [44]:
data_set = [[2, 3], [5,4], [9,6], [4,7], [8,1], [7,2]]
kd = kdTree(data_set)
preorder(kd.root)

[7, 2]
[5, 4]
[2, 3]
[4, 7]
[9, 6]
[8, 1]


习题3.2

In [36]:
train_data = [[2,3], [5,4], [9,6], [4,7], [8,1], [7,2]]
kd = kdTree(train_data)
point = [3, 4, 5]

In [37]:
res = find_nearest(kd, point)

In [38]:
res

Result_tuple(nearest_point=[2, 3], nearest_dist=1.4142135623730951, nodes_visited=4)

习题3.3

参照算法3.3, 写出输出为x的k临近算法

In [42]:
from collections import namedtuple
import numpy as np

In [ ]:
result = namedtuple('result_tuple', "nearest_point nearest_dist nodes_d_visted")

In [47]:
class kdNode(object):
    def __init__(self):
        self.element = element
        self.split = split
        self.left = left
        self.right = right
        

class kd_tree(object):

    def __init__(self, data):
        k = len(data[0])
        
        def CreateNode(split, data):
            if not data:
                return None
            else:
                split_pos = len(data) // 2
                data_set = data.sort(key = lambda x: x[split_pos])
                median = data_set[split_pos]
                next_split = (split + 1) % k
            return kdNode(
                median,
                split,
                CreateNode(split_next, data[:split_pos + 1]),
                CreateNode(plit_next, data[split_pos:])
            )
        self.root = CreateNode(0, data)
        
    def search(self, kdNode, target_point, max_dist):
        nodes_visited = 0
        
        if self.root == None:
            return result([0] * k, float("inf"), 0)
        
        s = self.root.split
        if target_point[s] < self.element[s]:
            nearest_point = self.left
            further_point = self.right
        else:
            nearest_point = self.right
            further_point = self.left
            
        nodes_visited += 1
        
        tmpNode = self.search(nearest_point, target_point, max_dist)  #走到叶节点
        
        nearest = tmpNode.nearest_point
        dist = tmpNode.nearest_dist
        
        if  dist < max_dist:         #这里写反了 为什么呢
            max_dist = dist
            
        temp_dist = abs(self.element[s] - target_point[s]) #如果和超平面相交， 那就有可能和其他子节点有最近距离
        if max_dist < temp_dist: #判断超球体是否与超平面相交
            return result(nearest, dist, nodes_visited) # 不相交可以直接返回 不用继续判断
        
        #计算目标点与分割点的欧氏距离
        temp_dist = sqrt(sum(p1 - p2)** 2 for p1, p2 in zip(pivot, target))
        if temp_dist < dist:
            nearest = pivot
            dist = temp_dist
            max_dist = dist
        temp2 = travel(further_node, target, max_dist)
        
        nodes_visited += temp2.nodes_visited
        if temp2.nearest_dist < dist:
            nearest = temp2.nearest_point
            dist = temp2.nearest_dist
        
        
        return result(nearest_point, nearest_dist, nodes_visited)
    return travel(tree.root, point, float("inf"))

In [8]:
class Fabs(object):
    
    def cal(self,num):
        if num > 1 :
            return self.cal(num-1) + self.cal(num-2)
        elif(num == 1): 
            return 1
        else: return 0

In [9]:
f = Fabs()

In [11]:
f.cal(10)

55